# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city= "./city.csv"
city_df = pd.read_csv(city)
city_df.head(5)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Configure gmaps
gmaps.configure(api_key=g_key)



In [57]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
weight = city_df["Humidity"].astype(float)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
reduced_df = city_df[city_df['Max Temp']>75]
reduced_df = reduced_df[reduced_df['Max Temp']<85]
reduced_df = reduced_df[reduced_df['Wind Speed']<10]
reduced_df = reduced_df[reduced_df['Cloudiness']<=5]
reduced_df.dropna()
reduced_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,Atuona,-9.80,-139.03,79.178,71,2,6.25,PF,1607890855
35,35,Kapaa,22.08,-159.32,78.008,78,1,3.10,US,1607890978
52,52,Pisco,-13.70,-76.22,75.200,64,0,9.30,PE,1607891124
61,61,Makkah al Mukarramah,21.43,39.83,75.020,60,0,1.02,SA,1607891120
89,89,Puerto Ayora,-0.74,-90.35,80.600,65,0,7.70,EC,1607891129
121,121,Vila Velha,-20.33,-40.29,82.400,58,0,7.20,BR,1607891136
123,123,Epe,6.58,3.98,78.512,90,3,2.64,NG,1607891136
175,175,Cabo San Lucas,22.89,-109.91,78.008,64,5,5.10,MX,1607891145
182,182,San Juan,-31.54,-68.54,81.626,21,0,4.89,AR,1607891146
185,185,Bubaque,11.28,-15.83,81.266,63,0,5.04,GW,1607891146


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = reduced_df
hotel_df['Hotel Name'] = ""

cities = hotel_df['City']
lats = hotel_df['Lat']
lons = hotel_df['Lng']

for x in hotel_df.index:
    
    # geocoordinates
    target_coordinates = "{" + str(lats[x]) + "," + str(lons[x]) + "}"
    target_search = cities[x]
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()
    print(places_data)
    # store the name of the first hotel that appears
    #hotel_df['Hotel Name'].append(places_data["results"][0]["name"])

hotel_df

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
